# Lognormal sampling for JEFF-3.3 W-184

In [ ]:
import sandy

In [ ]:
import pandas as pd

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
endf6 = sandy.get_endf6_file("jeff_33", "xs", 741840)

In [ ]:
mts = [4] + list(range(51, 69)) + [91]
cov = endf6.get_errorr(err=1, chi=False, errorr33_kws=dict(ign=13), mubar=False, verbose=True)["errorr33"].get_cov()

In [ ]:
s = cov.get_std().reset_index().query("MT in @mts")
s["ERIGHT"] = [x.right for x in s.E]
s["MT"] = s["MT"].astype("category")
s["STD"] *= 100

fig, ax = plt.subplots(figsize=(8, 4), dpi=100)
sns.lineplot(data=s, x="ERIGHT", y="STD", ax=ax, hue="MT", drawstyle="steps-pre", palette="tab20")
ax.set(xlim=(1e5, 2e7), xscale="log", ylim=[0, 200], ylabel="standard deviation / $\%$", xlabel="energy / $eV$")
ax.legend(ncol=2)
fig.tight_layout()

In [ ]:
nsmp = 1000
s_lognorm = cov.sampling(nsmp, pdf="lognormal")
s_norm = cov.sampling(nsmp, pdf="normal")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi=100)

s = s_lognorm.get_rstd().reset_index().query("MT in [4, 91]")
s["ERIGHT"] = [x.right for x in s.E]
s["MT"] = s["MT"].astype("category")
s["RSTD"] *= 100
s = s.assign(what="lognormal")
sln = s

s = s_norm.get_rstd().reset_index().query("MT in [4, 91]")
s["ERIGHT"] = [x.right for x in s.E]
s["MT"] = s["MT"].astype("category")
s["RSTD"] *= 100
s = s.assign(what="normal")
sn = s

s = cov.get_std().reset_index().query("MT in [4, 91]")
s["ERIGHT"] = [x.right for x in s.E]
s["MT"] = s["MT"].astype("category")
s["RSTD"] = s["STD"] * 100
s = s.assign(what="reference")

s = pd.concat([sn, sln, s], ignore_index=True)

sns.lineplot(data=s, x="ERIGHT", y="RSTD", ax=ax, hue="MT", drawstyle="steps-pre", palette="tab20", ls="-", style="what")
ax.set(xlim=(1e5, 2e7), xscale="log", ylim=[0, 200], ylabel="standard deviation / $\%$", xlabel="energy / $eV$")
ax.legend(ncol=2)
fig.tight_layout()